<a href="https://colab.research.google.com/github/SarathChandran/amazon-sagemaker-examples/blob/master/Finetune_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
import pprint

# locale.getpreferredencoding = lambda: "UTF-8"
# locale.getpreferredencoding = lambda: "UTF-8"
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"


locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q bitsandbytes einops safetensors torch xformers datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requiremen

In [3]:
# !pip install -q -U bitsandbytes einops safetensors torch xformers datasets
# !pip install -q -U transformers peft accelerate

In [4]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

In [5]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
tokenizer.eos_token

'<|endoftext|>'

In [8]:
!pip list | grep bits

bitsandbytes                     0.41.1


In [9]:
import accelerate

accelerate.__version__

'0.23.0.dev0'

In [10]:
import bitsandbytes

accelerate.__version__

'0.23.0.dev0'

In [11]:
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
  )

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [12]:
from peft import prepare_model_for_kbit_training

pretrained_model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(pretrained_model)

In [15]:
def print_trainable_parameters(model):

  trainable_params = 0
  all_params = 0
  for _, param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
  print(f"trainable params: {trainable_params} || all params: {all_params} || trainable % {100 * trainable_params/all_params}")

In [16]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable % 0.13058363808693696


### Prepare Data

In [17]:
from datasets import load_dataset

data = load_dataset("celikmus/mayo_clinic_symptoms_and_diseases_v1", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [18]:
data

Dataset({
    features: ['text', 'label'],
    num_rows: 1058
})

In [19]:
tokenizer.pad_token = tokenizer.eos_token

train_dataset = data.map(
    lambda x: {
        "input_text": f"symptoms: {x['text']}; most likely explaination: {x['label']}"
    }
)

train_encodings = tokenizer(
    train_dataset["input_text"],
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="pt"
)

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [34]:
class TextDataset(Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = item["input_ids"].clone()
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])


In [35]:
train_dataset = TextDataset(train_encodings)

In [36]:
train_encodings

{'input_ids': tensor([[   94, 21623,  3702,  ...,   248,  7038,  1001],
        [   94, 21623,  3702,  ...,    11,    11,    11],
        [   94, 21623,  3702,  ...,    11,    11,    11],
        ...,
        [   94, 21623,  3702,  ...,    11,    11,    11],
        [   94, 21623,  3702,  ...,   193, 20222, 11331],
        [   94, 21623,  3702,  ...,    11,    11,    11]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [37]:
train_dataset

In [38]:
request_text = """
In crowded places, I feel cold in the tips of my fingers, I sweat with dizziness.
What is happening?
"""

In [39]:
encoding = tokenizer(request_text, return_tensors="pt").to("cuda:0")
pretrained_model_output = pretrained_model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.25,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [40]:
pprint.pprint(tokenizer.decode(pretrained_model_output[0], skip_special_tokens=True))

('\n'
 'In crowded places, I feel cold in the tips of my fingers, I sweat with '
 'dizziness.\n'
 'What is happening?\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure.\n'
 'I am not sure')


In [41]:
request_text = "I started to feel swelling  and itching around the mouth and throat after a salad with peanuts, cherry tomatoes and cheese. What may be the reason?"
encoding = tokenizer(request_text, return_tensors="pt").to("cuda:0")
pretrained_model_output = pretrained_model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.25,
    eos_token_id=tokenizer.eos_token_id,
)
pprint.pprint(tokenizer.decode(pretrained_model_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


('I started to feel swelling  and itching around the mouth and throat after a '
 'salad with peanuts, cherry tomatoes and cheese. What may be the reason?\n'
 'It is possible that the peanuts, cherry tomatoes, and cheese may have caused '
 'an allergic reaction. It is recommended to avoid these foods if you '
 'experience symptoms such as swelling and itching.')


### Training

In [42]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=10,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=40,
        learning_rate=2.5e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type="cosine"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [43]:
trainer.train()

<ipython-input-34-81a1f9482024>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.211700
2,2.182800
3,2.110500
4,2.141200
5,2.042800
6,2.134900
7,2.069200
8,2.052000
9,1.984800
10,2.071700


TrainOutput(global_step=40, training_loss=1.8577986359596252, metrics={'train_runtime': 922.1589, 'train_samples_per_second': 1.388, 'train_steps_per_second': 0.043, 'total_flos': 6492863730941952.0, 'train_loss': 1.8577986359596252, 'epoch': 1.2})

### Example after fine tuning

In [46]:
trained_model = (
    trainer.model.module if hasattr(trainer.model, "module") else trainer.model
)
trained_model.save_pretrained("outputs")

In [47]:
lora_config = LoraConfig.from_pretrained("outputs")
loaded_model = get_peft_model(
    prepare_model_for_kbit_training(pretrained_model), lora_config
)

In [48]:
loaded_model.config.use_cache = True # disable for training, enable for inference
loaded_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_

In [49]:
del model
del trained_model
del trainer
import gc

gc.collect()

112

In [51]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"": 0}, trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [74]:
def generate_answer(query):
  system_prompt = """
  Answer the following question truthfully.
  If you don't know the answer or the question is too complex,
  respond 'Kindly, consult a doctor for further queries.'
  """
  user_prompt = f"""
  <HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-" * 50

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  pprint.pprint(encoding.attention_mask)
  output = pretrained_model.generate(
      input_ids=encoding.input_ids,
      attention_mask=encoding.attention_mask,
      max_new_tokens=100,
      do_sample=True,
      temperature=0.25,
      repetition_penalty=1.3,
      eos_token_id=tokenizer.eos_token_id,
  )

  text_output = tokenizer.decode(output[0], skip_special_tokens=True)

  pprint.pprint(dashline)
  pprint.pprint(f" ORIGINAL MODEL RESPONSE: \n {text_output}")
  pprint.pprint(dashline)

  peft_encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  pprint.pprint(peft_encoding.attention_mask)
  peft_output = loaded_model.generate(
      input_ids=peft_encoding.input_ids,
      attention_mask=peft_encoding.attention_mask,
      max_new_tokens=100,
      do_sample=True,
      temperature=0.25,
      repetition_penalty=1.3,
      eos_token_id=tokenizer.eos_token_id,
  )

  peft_text_output = tokenizer.decode(peft_output[0], skip_special_tokens=True)

  pprint.pprint(f" PEFT MODEL RESPONSE: \n {peft_text_output}")
  pprint.pprint(dashline)

In [75]:
query = """In crowded places, I feel cold in the tips of my fingers, I sweat with dizziness. What may be happening?"""
generate_answer(query)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'--------------------------------------------------'
(' ORIGINAL MODEL RESPONSE: \n'
 ' \n'
 '  Answer the following question truthfully.\n'
 "  If you don't know the answer or the question is too complex,\n"
 "  respond 'Kindly, consult a doctor for further queries.'\n"
 '  \n'
 '\n'
 '  <HUMAN>: In crowded places, I feel cold in the tips of my fingers, I sweat '
 'with dizziness. What may be happening?\n'
 '  <ASSISTANT>: <HUMAN> may be experiencing symptoms of hypothermia. It is '
 'important to seek medical attention immediately.')
'--------------------------------------------------'
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')
(' PEFT MODEL RESPONSE: \n'
 ' \n'
 '  Answer the following question truthfully.\n'
 "  If you don't know the answer or the q

In [76]:
query = """I started to feel swelling  and itching around the mouth and throat after a salad with peanuts, cherry tomatoes and cheese. What may be the reason?"""
generate_answer(query)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'--------------------------------------------------'
(' ORIGINAL MODEL RESPONSE: \n'
 ' \n'
 '  Answer the following question truthfully.\n'
 "  If you don't know the answer or the question is too complex,\n"
 "  respond 'Kindly, consult a doctor for further queries.'\n"
 '  \n'
 '\n'
 '  <HUMAN>: I started to feel swelling  and itching around the mouth and '
 'throat after a salad with peanuts, cherry tomatoes and cheese. What may be '
 'the reason?\n'
 '  <ASSISTANT>: <HUMAN> may be allergic to peanuts, cherry tomatoes or '
 "cheese. It's best to consult a doctor for further diagnosis and treatment.")
'--------------------------------------------------'
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')
(' PEFT MODEL RESPONSE: \n'
 ' \n'
 ' 